In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 18.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [7]:
#zdefiniowanie modelu
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',min_value=1,max_value=3)):
        model.add(layers.Flatten(input_shape=(28,28)))
        model.add(layers.Dense(units=hp.Int('units_'+str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(10,activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(
        hp.Choice('learning_rate',
                  values=[1e-2,1e-3,1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [8]:
#inicjalizacja strojenia hiperparametrów
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='myparams'
)

Reloading Tuner from my_dir/myparams/tuner0.json


In [9]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train,x_test = x_train/255.0,x_test/255.0

In [10]:
tuner.search(x_train,y_train,
             epochs=5,
             validation_data=(x_test,y_test))

Trial 5 Complete [00h 05m 40s]
val_accuracy: 0.9419333140055338

Best val_accuracy So Far: 0.9746333360671997
Total elapsed time: 00h 19m 16s


In [11]:
best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()

Results summary
Results in my_dir/myparams
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 3
units_0: 96
learning_rate: 0.001
units_1: 192
units_2: 480
units_3: 352
Score: 0.9746333360671997

Trial 3 summary
Hyperparameters:
num_layers: 3
units_0: 160
learning_rate: 0.0001
units_1: 256
units_2: 256
units_3: 384
Score: 0.9706999858220419

Trial 4 summary
Hyperparameters:
num_layers: 5
units_0: 320
learning_rate: 0.01
units_1: 416
units_2: 128
units_3: 288
units_4: 32
Score: 0.9419333140055338

Trial 0 summary
Hyperparameters:
num_layers: 4
units_0: 480
learning_rate: 0.0001
units_1: 32
units_2: 32
units_3: 32
Traceback (most recent call last):
  File "/opt/python/envs/default/lib/python3.8/site-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/opt/python/envs/default/lib/python3.8/site-packages/keras_tuner/src/e

In [12]:
eval_loss,eval_accuracy = best_model.evaluate(x_test,y_test)
print(f'Najlepszy model-> Loss: {eval_loss}, Accuracy: {eval_accuracy}')

313/313 [==============================] - 3s 7ms/step - loss: 0.0834 - accuracy: 0.9764
Najlepszy model-> Loss: 0.08335208147764206, Accuracy: 0.9764000177383423
